# OpenAI Gym
[Gym](https://www.gymlibrary.dev/) is an open-source Python library, made for the easy development and testing of reinforcement learning algorithms. Today we will use it to create and run an instance of the Atari game Breakout. The Gym library provies us access to the game state, game rewards, and available actions, which if you remember are nesseicary parts of our RL framework. 

<img src="Test.gif" width="200" align="center">

### The RL Framework for Breakout:
- **Action:** Move the paddle left and right 
- **State:** The 210x160 RGB image frame 
- **Reward:** Amount the game score increases 


### Packages

In [1]:
import gym 
import cv2
import time 
import imageio

We need to set the environment variable ALE_PY_ROM_DIR to the directory of the bins so that we can use the namespace ALE/

In [2]:
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import Breakout
ale.loadROM(Breakout)

A.L.E: Arcade Learning Environment (version 0.8.0+919230b)
[Powered by Stella]
Game console created:
  ROM file:  /Users/justinvalentine/opt/anaconda3/envs/Test/lib/python3.9/site-packages/ale_py/roms/breakout.bin
  Cart Name: Breakout - Breakaway IV (1978) (Atari)
  Cart MD5:  f34f08e5eb96e500e851a80be3277a56
  Display Format:  AUTO-DETECT ==> NTSC
  ROM Size:        2048
  Bankswitch Type: AUTO-DETECT ==> 2K

Running ROM file...
Random seed is 1667373141


## Creating the Atari Breakout Instance 

In [8]:
env = gym.make("ALE/Breakout-v5") # creats a game instance of Atari Breakout 

env.reset()
image_lst = []

step_num, total_reward = 0, 0

while step_num < 100:
    # Used to generates random actions
    action = env.env.action_space.sample()
    
    # Get the next state, and reward after taking your action 
    state, reward, terminated, truncated, info = env.step(action)

    # Create image frame 
    img = cv2.resize(state, (160, 210), interpolation=cv2.INTER_CUBIC)
    frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_lst.append(frame)
        
    time.sleep(0.01)
    step_num += 1

env.close()

imageio.mimsave('random-action.gif', image_lst, fps=30)

<img src="random-action.gif" width="300" align="center">

## Applying Reinforcement Learning to Breakout 
Recall that the main idea of Reinforcemnt Learnign is for our Agent to select the best action at each time step given its current state at that time step. When we say 'best action' we mean that the agent is making the action with the goal of maximising total reward.  

However it is not enough for the agent to just select the greedy action at every time-step, the agent must also explore its other options. The agent then 'learns' what actions are good in what states, and 'remembers' these resutls. 

So how can we train the Agent to play like a human, when all the agent can do interact whith the game and see how it affects the pixels?

### The Challenges
The first challenge we need to addres is that each state of the game contains alot of information! 
In this case, if we have a video game with a 640x480 resolution, the current state of a game frame will be encoded by 640x480=307200 pixels values, and the number of possible state variations will be enormous